In [4]:
import json
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
#from tweepy.streaming import StreamListener
from credentials import *

In [11]:
"""Establecer acceso a la API de Twitter."""
CONSUMER_KEY = "uvcMjAH4FJglTG8QS9zlpJC01"
CONSUMER_SECRET = 'Ed2c2FhXzpHFagbfTVb3gTnziNTQ3hDk0UDRiiraPfupO5At0s'
ACCESS_TOKEN = "1442475234913775619-hKJkC4Pp3N0ztkvGAue6niVGtra8YZ"
ACCESS_SECRET = "QqW0LjrlYr1YrkllT1RhMbi4LmUvARaecltG6HTzAobUf"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth)
print("authenticated")

authenticated


In [7]:
tampico_madero = [-97.87777, 22.28519, -96.87777, 23.28519, -97.83623, 22.27228, -96.83623, 23.27228]

CONVERTIR TWEETS

In [12]:
import pandas as pd
import numpy as np

In [39]:
tweets_raw = pd.read_table('C:/Users/jdroj/Downloads/BigData/TweetsMap/tweets_tampico_madero.csv', header = None, iterator=False)
tweets_2 = pd.DataFrame()
tweets_raw

,0
0,"2020-07-22 16:43:58-05:00,@dnkdxpe,@yedidgdr T..."
1,"2020-07-22 16:45:11-05:00,@laura_armz,Otro día..."
2,"2020-07-22 16:45:22-05:00,@_UnJaibo,Entro a IG..."
3,"2020-07-22 16:46:53-05:00,@JoseWiedmer,Mi abue..."
4,"2020-07-22 16:50:47-05:00,@taniakarinagon,Quie..."
...,...
2044,"2020-08-20 15:10:09-05:00,@theuniongap,@Marcel..."
2045,"2020-08-20 15:11:14-05:00,@ViVaMaderV,@lebudin..."
2046,"2020-08-20 15:12:34-05:00,@MartinMaggot23,@Die..."
2047,"2020-08-20 15:13:26-05:00,@gabs_nmo,""Mood de h..."


In [ ]:
#stamp = Timestamp('1/1/2014 16:20', tz='America/Sao_Paulo')
#new_stamp = stamp.tz_convert('US/Eastern')

In [40]:
while 1:
    #Debido a que el archivo de texto de los tweets es demasiado grande
    #debe de ser procesado por chunks en lugar de cargar todo el archivo en memoria
    tweets = tweets_raw.get_chunk(500) #1000 filas por chunk
    tweets.columns = ['tweets']
    tweets['len'] = tweets.tweets.apply(lambda x: len(x.split('|')))
    tweets[tweets.len < 4] = np.nan
    del tweets['len']
    tweets = tweets[tweets.tweets.notnull()]    
    
    #Establecer las columnas y valores que tomarán
    tweets['user'] = tweets.tweets.apply(lambda x: x.split('|')[0])
    tweets['geo'] = tweets.tweets.apply(lambda x: x.split('|')[1])
    tweets['timestamp'] = tweets.tweets.apply(lambda x: x.split('|')[2])
    tweets['tweet'] = tweets.tweets.apply(lambda x: x.split('|')[3])
    tweets['lat'] = tweets.geo.apply(lambda x: x.split(',')[0].replace('[',''))
    tweets['lon'] = tweets.geo.apply(lambda x: x.split(',')[1].replace(']',''))
    del tweets['tweets']
    del tweets['geo']
    
    #Convertir las latitudes y longitudes de string a float
    tweets['lon'] = pd.to_numeric(tweets['lon'], downcast="float")
    tweets['lat'] = pd.to_numeric(tweets['lat'], downcast="float")
    
    #Cambiar la zona horaria de UTC a GMT-5
    tweets['timestamp'] = pd.to_datetime(tweets['timestamp'], utc = False)
    stamp = tweets['timestamp']
    #tweets = tweets.set_index('timestamp').dt.tz_convert('America/Mexico_City').reset_index()
    tweets = tweets['timestamp'].dt.tz_localize('UTC').dt.tz_convert('America/Mexico_City')
    #Almacenar los tweets en el dataframe
    tweets_2 = tweets_2.append(tweets, ignore_index = True)
    #Guardar los tweets en un archivo CSV
    tweets.to_csv('C:/Users/jdroj/Downloads/BigData/TweetsMap/tweets_tampico_madero.csv', mode='a', header=False, index=False)


AttributeError: 'DataFrame' object has no attribute 'get_chunk'